In [3]:
import pandas as pd
import os
    
datasets = {"Retail_Rocket": {"path": '/home/jy1559/Mar2025_Module/Datasets/Retail_Rocket',
                              "file_names": ['category_tree.csv', 'events.csv', 'item_properties_part1.csv', 'item_properties_part2.csv']},
            "Diginetica": {"path": '/home/jy1559/Mar2025_Module/Datasets/Diginetica',
                              "file_names": ['product-categories.csv', 'products.csv', 'train-clicks.csv', 'train-item-views.csv', 'train-purchases.csv', 'train-queries.csv']},
            "LFM-BeyMS": {"path": '/home/jy1559/Mar2025_Module/Datasets/LFM-BeyMS/dataset',
                              "file_names": ['beyms.csv', 'events.csv', 'genre_annotations.csv', 'mainstreaminess.csv', 'ms.csv', 'user_groups.csv']},
            "Beauty": {"path": '/home/jy1559/Mar2025_Module/Datasets/Amazon',
                "file_names": ['All_Beauty.jsonl', 'meta_All_Beauty.jsonl']},
            "Game": {"path": '/home/jy1559/Mar2025_Module/Datasets/Amazon',
                "file_names": ['Video_Games.jsonl', 'meta_Video_Games.jsonl']},
            "MIND": {"path": {"train": '/home/jy1559/Mar2025_Module/Datasets/MIND/MINDlarge_train',
                              "val": '/home/jy1559/Mar2025_Module/Datasets/MIND/MINDlarge_dev',
                              "test": '/home/jy1559/Mar2025_Module/Datasets/MIND/MINDlarge_test'},
                "file_names": ['behavior.tsv', 'entity_embedding.vec', 'news.tsv', 'relation_embedding.vec']}}
dataset = datasets["MIND"]
directory_path = dataset['path']["train"]
file_names = dataset["file_names"]

behavior_path = os.path.join(directory_path, dataset["file_names"][0])
entity_path = os.path.join(directory_path, dataset["file_names"][1])
news_path = os.path.join(directory_path, dataset["file_names"][2])
relation_path = os.path.join(directory_path, dataset["file_names"][3])

for name in file_names:
    file_path = os.path.join(directory_path, name)
    try:
        if 'csv' in file_path:
            df = pd.read_csv(file_path)
        elif 'json' in file_path:
            df = pd.read_json(file_path, lines=True)
        else: df = None
        if df:
            print(f"First 5 rows of {name}:")
            print(df.head(), "\n")
    except FileNotFoundError:
        print(f"File {name} not found in the directory {directory_path}.")
    except pd.errors.EmptyDataError:
        print(f"File {name} is empty.")
    except pd.errors.ParserError:
        print(f"Error parsing {name}. Please check the file for inconsistencies.")

In [ ]:
import os
import json
import glob

def process_files(folder):
    # 기사 메타데이터: key = article id, value = 요약 텍스트
    item_metadata = {}
    # 사용자 상호작용 임시 저장: key = user id, value = list of interaction events
    user_interactions = {}

    # 폴더 내 20170101 ~ 20170107 파일 (jsonl 형식) 읽기
    file_pattern = os.path.join(folder, "2017010[1-7]*")
    files = sorted(glob.glob(file_pattern))
    
    for file_path in files:
        print(f"Processing file: {file_path}")
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    record = json.loads(line)
                except Exception as e:
                    print(f"Error parsing line: {e}")
                    continue

                # interaction 및 article view를 위해서는 "profile" 필드가 존재해야 함
                if "profile" not in record:
                    continue

                # article id 추출: 우선 "id", 없으면 "canonicalUrl"
                article_id = record.get("id")
                if not article_id:
                    article_id = record.get("canonicalUrl")
                if not article_id:
                    continue  # 기사 id 없으면 skip

                # item metadata: 이미 저장되어 있지 않은 경우, 필요한 텍스트 정보만 한 문장으로 요약
                if article_id not in item_metadata:
                    # 사용 가능한 텍스트 필드: title, category1, keywords, author
                    title = record.get("title", "").strip()
                    category = record.get("category1", "").strip()
                    keywords = record.get("keywords", "").strip()
                    authors = record.get("author", [])
                    if isinstance(authors, list):
                        authors = ", ".join(authors)
                    else:
                        authors = str(authors).strip()
                    
                    # 간단 요약문 구성 (너무 길지 않도록, 필요 시 길이 제한 적용)
                    summary_parts = []
                    if title:
                        summary_parts.append(title)
                    if category:
                        summary_parts.append(category)
                    if keywords:
                        summary_parts.append(keywords)
                    if authors:
                        summary_parts.append(authors)
                    summary_text = ". ".join(summary_parts)
                    # 예: 길이 제한(200자)
                    summary_text = summary_text[:200]
                    item_metadata[article_id] = summary_text

                # user interaction: user id, 타임스탬프, 기사 id, 그리고 부가정보
                user_id = record.get("userId")
                if not user_id:
                    continue

                interaction = {
                    "article_id": article_id,
                    "timestamp": record.get("time"),
                    "city": record.get("city", ""),
                    "deviceType": record.get("deviceType", ""),
                    "os": record.get("os", ""),
                    # session boundary flag: 추후 세션 분할에 사용
                    "sessionStart": record.get("sessionStart", False),
                    "sessionStop": record.get("sessionStop", False)
                }
                if user_id not in user_interactions:
                    user_interactions[user_id] = []
                user_interactions[user_id].append(interaction)

    # 세션 기준으로 상호작용을 그룹화: 각 사용자의 상호작용을 timestamp 순으로 정렬한 뒤,
    # sessionStart가 True이면 새로운 세션 시작, sessionStop가 True이면 해당 세션 종료.
    user_sessions = {}
    for user, interactions in user_interactions.items():
        # 타임스탬프 기준 정렬
        interactions_sorted = sorted(interactions, key=lambda x: x["timestamp"] or 0)
        sessions = []
        current_session = []
        for inter in interactions_sorted:
            # sessionStart 플래그가 True면, 현재 세션이 있다면 저장 후 새로 시작
            if inter.get("sessionStart", False):
                if current_session:
                    sessions.append(current_session)
                    current_session = []
            # 상호작용 기록(불필요한 플래그는 제외)
            current_session.append({
                "article_id": inter["article_id"],
                "timestamp": inter["timestamp"],
                "city": inter["city"],
                "deviceType": inter["deviceType"],
                "os": inter["os"]
            })
            # sessionStop 플래그가 True면 현재 세션 종료
            if inter.get("sessionStop", False):
                sessions.append(current_session)
                current_session = []
        # 남은 세션이 있다면 추가
        if current_session:
            sessions.append(current_session)
        user_sessions[user] = sessions

    return item_metadata, user_sessions

if __name__ == "__main__":
    # 데이터 파일이 들어 있는 폴더 경로 (예: "./data")
    folder = '/home/jy1559/Mar2025_Module/Datasets/Adressa'
    item_metadata, user_sessions = process_files(folder)
    
    # 결과를 각각의 JSON 파일로 저장
    with open("item_metadata.json", "w", encoding="utf-8") as f:
        json.dump(item_metadata, f, ensure_ascii=False, indent=2)
    print("Saved item_metadata.json")
    
    with open("user_interactions.json", "w", encoding="utf-8") as f:
        json.dump(user_sessions, f, ensure_ascii=False, indent=2)
    print("Saved user_interactions.json")


/tmp/ipykernel_3676813/204098059.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  item_props = df_props_latest.groupby('itemid').apply(aggregate_properties).to_dict()
100%|██████████| 417053/417053 [00:05<00:00, 81072.33it/s]


Item sentences JSON이 'retailrocket_item_sentences.json' 파일로 저장되었습니다.


In [4]:
import pandas as pd
import json

# 파일 경로 (실제 경로에 맞게 수정)
events_path = os.path.join(directory_path, dataset["file_names"][1])
df_events = pd.read_csv(events_path)

# timestamp를 datetime으로 변환 (Retail Rocket의 timestamp는 밀리초초 단위라고 가정)
df_events['datetime'] = pd.to_datetime(df_events['timestamp'], unit='ms')
df_events = df_events.sort_values(by=['visitorid', 'datetime'])

# 필요한 정보만 추출: (itemid, timestamp, event, transactionid)
def extract_event(row):
    return {
        "itemid": row["itemid"],
        "timestamp": row["datetime"].strftime('%Y-%m-%d %H:%M:%S'),
        "event": row["event"],
        "transactionid": row["transactionid"] if pd.notna(row["transactionid"]) else ""
    }

df_events["event_info"] = df_events.apply(extract_event, axis=1)

# 사용자(visitorid)별로 그룹화하고, 시간 간격이 3시간(10800초) 이상이면 세션 분리
user_sessions = {}
three_hours = pd.Timedelta(seconds=10800)

for visitor, group in tqdm(df_events.groupby('visitorid')):
    group = group.sort_values('datetime')
    sessions = []
    current_session = []
    prev_time = None
    for _, row in group.iterrows():
        current_time = row['datetime']
        if prev_time is not None and (current_time - prev_time) >= three_hours:
            if current_session:
                sessions.append(current_session)
            current_session = []
        current_session.append(row["event_info"])
        prev_time = current_time
    if current_session:
        sessions.append(current_session)
    user_sessions[visitor] = sessions

# JSON 파일로 저장 (각 사용자별 세션을 포함)
with open("retailrocket_user_interactions.json", "w", encoding="utf-8") as f:
    json.dump(user_sessions, f, ensure_ascii=False, indent=2)

print("User interactions JSON이 'retailrocket_user_interactions.json' 파일로 저장되었습니다.")


100%|██████████| 1407580/1407580 [05:50<00:00, 4010.43it/s]


User interactions JSON이 'retailrocket_user_interactions.json' 파일로 저장되었습니다.


In [12]:
import json

# 기존 interaction JSON 파일 로드
with open("retailrocket_user_interactions.json", "r", encoding="utf-8") as f:
    user_sessions = json.load(f)

filtered_user_sessions = {}

for user, sessions in user_sessions.items():
    total_interactions = sum(len(session) for session in sessions)
    # 사용자별 세션 수가 3개 이상이고, 총 interaction 수가 5회 이상인 사용자만 선택
    if len(sessions) >= 2 and total_interactions >= 3:
        filtered_user_sessions[user] = sessions

# 필터링 결과를 새로운 JSON 파일로 저장
with open("retailrocket_user_interactions_filtered.json", "w", encoding="utf-8") as f:
    json.dump(filtered_user_sessions, f, ensure_ascii=False, indent=2)

print("Filtered interactions saved to 'retailrocket_user_interactions_filtered.json'.")


Filtered interactions saved to 'retailrocket_user_interactions_filtered.json'.


In [5]:
import json
import statistics

# JSON 파일 읽기
with open("retailrocket_user_interactions.json", "r", encoding="utf-8") as f:
    user_sessions = json.load(f)

# 사용자 수
num_users = len(user_sessions)

# 사용자별 세션 수와 전체 세션별 interaction 수 목록 계산
session_counts = []         # 각 사용자별 세션 수
interaction_counts = []     # 각 세션의 interaction 수

for user, sessions in tqdm(user_sessions.items()):
    session_counts.append(len(sessions))
    for session in sessions:
        interaction_counts.append(len(session))

# 사용자당 세션 수의 평균 계산
avg_sessions_per_user = sum(session_counts) / num_users if num_users > 0 else 0

# 각 세션당 interaction 수의 평균과 분산 계산
if interaction_counts:
    avg_interactions = statistics.mean(interaction_counts)
    var_interactions = statistics.variance(interaction_counts) if len(interaction_counts) > 1 else 0
else:
    avg_interactions = 0
    var_interactions = 0

print("Number of users:", num_users)
print("Average sessions per user:", avg_sessions_per_user)
print("Average interactions per session:", avg_interactions)
print("Variance of interactions per session:", var_interactions)


100%|██████████| 1407580/1407580 [00:00<00:00, 3024449.11it/s]


Number of users: 1407580
Average sessions per user: 1.196835703832109
Average interactions per session: 1.6360158419414925
Variance of interactions per session: 11.471367550718051


In [13]:
import json
import statistics

# JSON 파일 읽기
with open("retailrocket_user_interactions_filtered.json", "r", encoding="utf-8") as f:
    user_sessions = json.load(f)

# 사용자 수
num_users = len(user_sessions)

# 사용자별 세션 수와 전체 세션별 interaction 수 목록 계산
session_counts = []         # 각 사용자별 세션 수
interaction_counts = []     # 각 세션의 interaction 수

for user, sessions in tqdm(user_sessions.items()):
    session_counts.append(len(sessions))
    for session in sessions:
        interaction_counts.append(len(session))

# 사용자당 세션 수의 평균 계산
avg_sessions_per_user = sum(session_counts) / num_users if num_users > 0 else 0

# 각 세션당 interaction 수의 평균과 분산 계산
if interaction_counts:
    avg_interactions = statistics.mean(interaction_counts)
    var_interactions = statistics.variance(interaction_counts) if len(interaction_counts) > 1 else 0
else:
    avg_interactions = 0
    var_interactions = 0

print("Number of users:", num_users)
print("Average sessions per user:", avg_sessions_per_user)
print("Average interactions per session:", avg_interactions)
print("Variance of interactions per session:", var_interactions)


100%|██████████| 96057/96057 [00:00<00:00, 1001074.04it/s]


Number of users: 96057
Average sessions per user: 3.2782618653507813
Average interactions per session: 2.679044140997142
Variance of interactions per session: 49.42560094597455
